In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# LangChain Agent Testing\n",
    "\n",
    "This notebook explores setting up and testing LangChain agents for business intelligence."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "import json\n",
    "from dotenv import load_dotenv\n",
    "\n",
    "# Add parent directory to path\n",
    "sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))\n",
    "\n",
    "# Load environment variables\n",
    "load_dotenv()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Setup LangChain"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from langchain.agents import initialize_agent, Tool, AgentType\n",
    "from langchain.llms import OpenAI\n",
    "from langchain.memory import ConversationBufferMemory\n",
    "from langchain.prompts import PromptTemplate\n",
    "from langchain.chains import LLMChain"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Mock Tools\n",
    "\n",
    "First, let's create some mock tools to simulate our data access."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Mock data\n",
    "mock_email_data = [\n",
    "    {\n",
    "        \"sender\": \"john@example.com\",\n",
    "        \"subject\": \"Feedback on Widget X\",\n",
    "        \"sentiment\": \"positive\",\n",
    "        \"main_issue\": \"Feature request\",\n",
    "        \"product\": \"Widget X\",\n",
    "        \"date\": \"2023-01-15\"\n",
    "    },\n",
    "    {\n",
    "        \"sender\": \"jane@example.com\",\n",
    "        \"subject\": \"Problem with order\",\n",
    "        \"sentiment\": \"negative\",\n",
    "        \"main_issue\": \"Shipping delay\",\n",
    "        \"product\": \"Widget Y\",\n",
    "        \"date\": \"2023-01-16\"\n",
    "    },\n",
    "    {\n",
    "        \"sender\": \"mike@example.com\",\n",
    "        \"subject\": \"Question about compatibility\",\n",
    "        \"sentiment\": \"neutral\",\n",
    "        \"main_issue\": \"Technical question\",\n",
    "        \"product\": \"Widget Z\",\n",
    "        \"date\": \"2023-01-17\"\n",
    "    }\n",
    "]\n",
    "\n",
    "mock_analytics_data = {\n",
    "    \"underperforming_pages\": [\n",
    "        {\n",
    "            \"page\": \"/products/widget-y\",\n",
    "            \"bounce_rate\": 75.2,\n",
    "            \"avg_session_duration\": 45.3,\n",
    "            \"performance_score\": 0.35\n",
    "        },\n",
    "        {\n",
    "            \"page\": \"/blog/post-2\",\n",
    "            \"bounce_rate\": 68.7,\n",
    "            \"avg_session_duration\": 62.1,\n",
    "            \"performance_score\": 0.38\n",
    "        }\n",
    "    ],\n",
    "    \"conversion_funnel\": {\n",
    "        \"overall_rate\": 2.8,\n",
    "        \"steps\": [\n",
    "            {\"step\": \"Homepage\", \"visitors\": 1000, \"conversion_rate\": 100.0},\n",
    "            {\"step\": \"Products Page\", \"visitors\": 450, \"conversion_rate\": 45.0},\n",
    "            {\"step\": \"Cart\", \"visitors\": 120, \"conversion_rate\": 26.7},\n",
    "            {\"step\": \"Checkout\", \"visitors\": 50, \"conversion_rate\": 41.7},\n",
    "            {\"step\": \"Thank You\", \"visitors\": 28, \"conversion_rate\": 56.0}\n",
    "        ]\n",
    "    }\n",
    "}\n",
    "\n",
    "# Mock tools\n",
    "def get_email_data(query):\n",
    "    \"\"\"Get email data based on a query.\"\"\"\n",
    "    if query.lower() == \"all\":\n",
    "        return json.dumps(mock_email_data, indent=2)\n",
    "    \n",
    "    if \"sentiment\" in query.lower():\n",
    "        sentiment_counts = {\"positive\": 0, \"neutral\": 0, \"negative\": 0}\n",
    "        for email in mock_email_data:\n",
    "            sentiment_counts[email[\"sentiment\"]] += 1\n",
    "        return json.dumps(sentiment_counts, indent=2)\n",
    "    \n",
    "    if \"product\" in query.lower():\n",
    "        products = {}\n",
    "        for email in mock_email_data:\n",
    "            product = email[\"product\"]\n",
    "            if product in products:\n",
    "                products[product] += 1\n",
    "            else:\n",
    "                products[product] = 1\n",
    "        return json.dumps(products, indent=2)\n",
    "    \n",
    "    return \"Query not understood. Try 'all', 'sentiment', or 'product'.\"\n",
    "\n",
    "def get_analytics_data(query):\n",
    "    \"\"\"Get analytics data based on a query.\"\"\"\n",
    "    if query.lower() == \"all\":\n",
    "        return json.dumps(mock_analytics_data, indent=2)\n",
    "    \n",
    "    if \"underperforming\" in query.lower() or \"pages\" in query.lower():\n",
    "        return json.dumps(mock_analytics_data[\"underperforming_pages\"], indent=2)\n",
    "    \n",
    "    if \"funnel\" in query.lower() or \"conversion\" in query.lower():\n",
    "        return json.dumps(mock_analytics_data[\"conversion_funnel\"], indent=2)\n",
    "    \n",
    "    return \"Query not understood. Try 'all', 'underperforming pages', or 'conversion funnel'.\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Define the Agent Tools"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define tools for the agent\n",
    "tools = [\n",
    "    Tool(\n",
    "        name=\"EmailData\",\n",
    "        func=get_email_data,\n",
    "        description=\"Get email analysis data. Query can be 'all', 'sentiment', or 'product'.\"\n",
    "    ),\n",
    "    Tool(\n",
    "        name=\"AnalyticsData\",\n",
    "        func=get_analytics_data,\n",
    "        description=\"Get website analytics data. Query can be 'all', 'underperforming pages', or 'conversion funnel'.\"\n",
    "    )\n",
    "]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize LLM\n",
    "llm = OpenAI(temperature=0)\n",
    "\n",
    "# Initialize memory\n",
    "memory = ConversationBufferMemory(memory_key=\"chat_history\")\n",
    "\n",
    "# Initialize agent\n",
    "agent = initialize_agent(\n",
    "    tools,\n",
    "    llm,\n",
    "    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,\n",
    "    memory=memory,\n",
    "    verbose=True\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test the Agent"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test with a simple query\n",
    "agent.run(\"What's the sentiment breakdown in our email data?\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test with a more complex query\n",
    "agent.run(\"What are our underperforming pages and what's our overall conversion rate?\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test asking for recommendations\n",
    "agent.run(\"Based on our data, what do you recommend we focus on improving?\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Create a Business Analysis Function"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def analyze_business_data(email_data, analytics_data):\n",
    "    \"\"\"Analyze business data and provide recommendations.\n",
    "    \n",
    "    Args:\n",
    "        email_data: Email analysis data\n",
    "        analytics_data: Website analytics data\n",
    "        \n",
    "    Returns:\n",
    "        Analysis and recommendations\n",
    "    \"\"\"\n",
    "    # Convert to strings if they're not already\n",
    "    if not isinstance(email_data, str):\n",
    "        email_data = json.dumps(email_data, indent=2)\n",
    "    \n",
    "    if not isinstance(analytics_data, str):\n",
    "        analytics_data = json.dumps(analytics_data, indent=2)\n",
    "    \n",
    "    # Create prompt\n",
    "    prompt = PromptTemplate(\n",
    "        input_variables=[\"email_data\", \"analytics_data\"],\n",
    "        template=\"\"\"\n",
    "        You are a business intelligence expert. Analyze the following data and provide insights and recommendations.\n",
    "        \n",
    "        EMAIL DATA:\n",
    "        {email_data}\n",
    "        \n",
    "        ANALYTICS DATA:\n",
    "        {analytics_data}\n",
    "        \n",
    "        Please provide:\n",
    "        1. A summary of key insights\n",
    "        2. Identified issues or opportunities\n",
    "        3. Specific recommendations for improvement\n",
    "        4. Suggested next steps\n",
    "        \n",
    "        Format your response in a clear, structured way with headings.\n",
    "        \"\"\"\n",
    "    )\n",
    "    \n",
    "    # Create chain\n",
    "    chain = LLMChain(llm=llm, prompt=prompt)\n",
    "    \n",
    "    # Run chain\n",
    "    result = chain.run(email_data=email_data, analytics_data=analytics_data)\n",
    "    \n",
    "    return result"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test the analysis function\n",
    "analysis = analyze_business_data(\n",
    "    get_email_data(\"all\"),\n",
    "    get_analytics_data(\"all\")\n",
    ")\n",
    "\n",
    "print(analysis)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}